In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import cv2
import matplotlib.pyplot as plt

In [ ]:
import cv2
import os
import glob
img_dir = '/kaggle/input/captcha-dataset/train_image/train' # Enter Directory of all images 
data_path = os.path.join(img_dir,'*.png')
files = glob.glob(data_path)
data = []    #contain train_img
for f1 in files:
    img = cv2.imread(f1)
    data.append(img)
train_wrds_label=[]   #contains train_img words name corresponding to train_img 
for dirname, _, filenames in os.walk('/kaggle/input/captcha-dataset/train_image/train'):
    for filename in filenames:
        train_wrds_label.append(filename.split(".")[0])

    # len(data)    

In [ ]:
fig, axs = plt.subplots(nrows = 5, ncols = 1, figsize = (5, 5))
for i in range(0, 5 ):
    ax = axs[i]
    ax.imshow(data[i])                
plt.show()

In [ ]:
len_wrds=[]
import os
for dirname, _, filenames in os.walk('/kaggle/input/captcha-dataset/train_image/train'):
    for filename in filenames:
        len_wrds.append(len(filename)-4)
print(len_wrds)     

In [ ]:
import os
dirname = 'train_img.png'
os.mkdir(dirname)

In [ ]:
contours_no=[]
idx=-1
for img in data:
    idx=idx+1
    img_rgb=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_gray=cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    _, thresh_img = cv2.threshold(img_gray, 127, 255, cv2.THRESH_BINARY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    morph_img = cv2.morphologyEx(thresh_img, cv2.MORPH_CLOSE, kernel)
    contours, hierarchy = cv2.findContours(
                                   image = morph_img, 
                                   mode = cv2.RETR_TREE, 
                                   method = cv2.CHAIN_APPROX_SIMPLE)
    
    contours_new=[]
    threshold_area = 1300    #threshold area    
    
    for cnt in contours:        
        area = cv2.contourArea(cnt)         
        if area > threshold_area:                   
             contours_new.append(cnt)
    
    # Draw the contour 
    img_copy = img_rgb.copy()
    final = cv2.drawContours(img_copy, contours_new, contourIdx = -1, 
                         color = (255, 0, 0), thickness = 2)

    
    img_copy = img_rgb.copy()
    
    cnt_rect=[]  #contour rectangles
    
    for c in contours_new:
        x,y,w,h=cv2.cv2.boundingRect(c)
        if(w<120 and h>65 and h<110):
            cnt_rect.append((x,y,w,h))
#             img_box = cv2.rectangle(img_copy, (x-5, y-5), (x+w+5, y+h+5), color = (255, 0, 0), thickness = 2)
#     removing the rectangles inside bigger rectangle leaving only the bigges rectangle
    
    rcr=[] #real contour rectangles afterremoving several inner rectangles from outer
    
    for i in cnt_rect:
        flag=0
        for j in cnt_rect:
                if((i[0]>j[0]) and (i[0]+i[2])<(j[0]+j[2]) and (i[1]>j[1]) and (i[1]+i[3])<(j[1]+j[3])):
                    flag=1
                    break
        if flag==0:
            rcr.append(i)
     
    rcr=sorted(rcr, key=lambda a: a[0]) #sorting contour rectangles wrt to x so that we get letters according the order on which they originally are  
    contours_no.append(len(rcr)) 
#     img_copy = img_rgb.copy()
#     for c in rcr:
#         (x,y,w,h)= c
#         img_box = cv2.rectangle(img_copy, (x-10, y-10), (x+w+10, y+h+10), color = (255, 0, 0), thickness = 2) 
    if contours_no[idx]==len_wrds[idx]: 
        letter_img=[]
        i=0
        for box in rcr:
            letter_img.append(img_copy[box[1]:box[1]+box[3],box[0]:box[0]+box[2]])
            img=letter_img[i]
#             img_i = Image.fromarray(letter_img[i], 'RGB')
#             img_i.save(os.path.join(dirname, train_wrds_label[idx] + str(i)), 'png')
            cv2.imwrite(os.path.join(dirname, train_wrds_label[idx] + str(i)), img)
            i=i+1
            
#     fig, axs = plt.subplots(nrows = 1, ncols = len(letter_img), figsize = (20, 20))
#     for i in range(0, len(letter_img) ):
#         ax = axs[i]
#         ax.imshow(letter_img[i])    

In [ ]:
print(contours_no)
# len(contours_no)

In [ ]:
#checking how much image have correct no of contour rectangles
c=0
d=0
f=0
idx=[]
for i in range(0,1500):
    if(contours_no[i]==len_wrds[i]):
        c=c+1
        idx.append(i)
    if(contours_no[i]>len_wrds[i]):
        d=d+1
#         idx.append(i)
    if(contours_no[i]<len_wrds[i]):
        f=f+1    
#         idx.append(i)
print(c,d,f,c+d+f)  
# print(idx)

In [ ]:
img = cv2.imread('train_img.png/AAE0')
plt.imshow(img)

In [ ]:
# #for deleting a directory
# import sys
# import shutil
# shutil.rmtree('test_img.png')

**Preparing training data**

In [ ]:
cnt = 0
acc = 0
x_train = []
y_train = []
label = 0
files = []
path = 'train_img.png/'
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        files.append(file)
        # print(file)
        gray = cv2.imread(path + file)
        gray = cv2.cvtColor(gray, cv2.COLOR_BGR2GRAY)
        gray = cv2.resize(255-gray, (100, 100))
        x_train.append(gray)
        y_train.append(file[int(file[-1])])
                

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)
print(x_train.shape)
print(y_train.shape)

In [ ]:
# print(y_train[10])
# plt.imshow(x_train[10],cmap='gray')

In [ ]:
y_train

In [ ]:
y_train=pd.get_dummies(y_train)

In [ ]:
y_train.isnull()

In [ ]:
y_train=pd.DataFrame(y_train).to_numpy()
x_train = x_train.reshape(x_train.shape[0], 100, 100, 1)
x_train = x_train.astype('float32')
x_train = x_train/255

In [ ]:
x_train.shape

In [ ]:
# print(x_train)
# print(y_train)

**TRAINING MODEL AND PREDICTING**

In [ ]:
# USING CNN
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers. normalization import BatchNormalization
import tensorflow as tf
import numpy as np

In [ ]:
model = Sequential()
model.add(Conv2D(50, kernel_size = (3, 3), activation='relu', input_shape=(100, 100, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size=(3,3), activation='softmax'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='softmax'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='softmax'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
# model.add(Conv2D(96, kernel_size=(3,3), activation='softmax'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
# model.add(Dense(256, activation='sigmoid'))
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(64, activation='sigmoid'))
#model.add(Dropout(0.3))
model.add(Dense(26, activation = 'sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [ ]:
model.fit(x_train, y_train, batch_size = 50, epochs = 10, verbose = 1)

In [ ]:
#preparing test data for calculation of accuracy of correct no of letters predicted(doing same stuff as before)
import cv2
import os
import glob
img_dir = '/kaggle/input/captcha-dataset/test_image/test' # Enter Directory of all images 
data_path = os.path.join(img_dir,'*.png')
files = glob.glob(data_path)
data = []    #contain train_img
for f1 in files:
    img = cv2.imread(f1)
    data.append(img)
test_wrds_label=[]   #contains train_img words name corresponding to train_img 
for dirname, _, filenames in os.walk('/kaggle/input/captcha-dataset/test_image/test'):
    for filename in filenames:
        test_wrds_label.append(filename.split(".")[0])

    # len(data)    


In [ ]:
len_wrds=[]
import os
for dirname, _, filenames in os.walk('/kaggle/input/captcha-dataset/test_image/test'):
    for filename in filenames:
        len_wrds.append(len(filename)-4)
print(len_wrds)     

In [ ]:
import os
dirname = 'test_img.png'
os.mkdir(dirname)

In [ ]:
contours_no=[]
idx=-1
for img in data:
    idx=idx+1
    img_rgb=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_gray=cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    _, thresh_img = cv2.threshold(img_gray, 127, 255, cv2.THRESH_BINARY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    morph_img = cv2.morphologyEx(thresh_img, cv2.MORPH_CLOSE, kernel)
    contours, hierarchy = cv2.findContours(
                                   image = morph_img, 
                                   mode = cv2.RETR_TREE, 
                                   method = cv2.CHAIN_APPROX_SIMPLE)
    
    contours_new=[]
    threshold_area = 1300    #threshold area    
    
    for cnt in contours:        
        area = cv2.contourArea(cnt)         
        if area > threshold_area:                   
             contours_new.append(cnt)
    
    # Draw the contour 
    img_copy = img_rgb.copy()
    final = cv2.drawContours(img_copy, contours_new, contourIdx = -1, 
                         color = (255, 0, 0), thickness = 2)

    
    img_copy = img_rgb.copy()
    
    cnt_rect=[]  #contour rectangles
    
    for c in contours_new:
        x,y,w,h=cv2.cv2.boundingRect(c)
        if(w<120 and h>65 and h<110):
            cnt_rect.append((x,y,w,h))
#             img_box = cv2.rectangle(img_copy, (x-5, y-5), (x+w+5, y+h+5), color = (255, 0, 0), thickness = 2)
#     removing the rectangles inside bigger rectangle leaving only the bigges rectangle
    
    rcr=[] #real contour rectangles afterremoving several inner rectangles from outer
    
    for i in cnt_rect:
        flag=0
        for j in cnt_rect:
                if((i[0]>j[0]) and (i[0]+i[2])<(j[0]+j[2]) and (i[1]>j[1]) and (i[1]+i[3])<(j[1]+j[3])):
                    flag=1
                    break
        if flag==0:
            rcr.append(i)
     
    rcr=sorted(rcr, key=lambda a: a[0]) #sorting contour rectangles wrt to x so that we get letters according the order on which they originally are  
    contours_no.append(len(rcr)) 
#     img_copy = img_rgb.copy()
#     for c in rcr:
#         (x,y,w,h)= c
#         img_box = cv2.rectangle(img_copy, (x-10, y-10), (x+w+10, y+h+10), color = (255, 0, 0), thickness = 2) 
    if contours_no[idx]==len_wrds[idx]: 
        letter_img=[]
        i=0
        for box in rcr:
            letter_img.append(img_copy[box[1]:box[1]+box[3],box[0]:box[0]+box[2]])
            img=letter_img[i]
#             img_i = Image.fromarray(letter_img[i], 'RGB')
#             img_i.save(os.path.join(dirname, train_wrds_label[idx] + str(i)), 'png')
            cv2.imwrite(os.path.join(dirname, test_wrds_label[idx] + str(i)), img)
            i=i+1
            
#     fig, axs = plt.subplots(nrows = 1, ncols = len(letter_img), figsize = (20, 20))
#     for i in range(0, len(letter_img) ):
#         ax = axs[i]
#         ax.imshow(letter_img[i])    

In [ ]:
cnt = 0
acc = 0
x_test = []
y_test = []
label = 0
files = []
i=0
path = 'test_img.png/'
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        files.append(file)
        # print(file)
        gray = cv2.imread(path + file)
        gray = cv2.cvtColor(gray, cv2.COLOR_BGR2GRAY)
        gray = cv2.resize(255-gray, (100, 100))
        x_test.append(gray)
        y_test.append(file[int(file[-1])])
        i=i+1        

In [ ]:
x_test = np.array(x_test)
y_test = np.array(y_test)
print(x_test.shape)
print(y_test.shape)


In [ ]:
y_test=pd.get_dummies(y_test)

In [ ]:
y_test.isnull()

In [ ]:
y_test=pd.DataFrame(y_test).to_numpy()
print(y_test)
# plt.imshow(x_test.reshape(x_test.shape[0],100,100))
x_test = x_test.reshape(x_test.shape[0], 100, 100, 1)
x_test = x_test.astype('float32')
x_test = x_test/255


In [ ]:
x_test[0].shape

In [ ]:
loss, acc = model.evaluate(x_test, y_test, verbose = 1)
print(acc * 100)

In [ ]:
y_pred = model.predict(x_test)
acc=0
print(len(y_pred))
for i in range(len(y_pred)):
#     print(chr(y_test[i].argmax()+65))
#     print(chr(y_pred[i].argmax()+65))
    if y_test[i].argmax()==y_pred[i].argmax():
        acc=acc+1

print(acc/len(y_pred))

In [ ]:
#calculation accuracy by counting total LCS(longest common subsequence)


In [ ]:
#preparing test data for calculation of accuracy of correct no of letters predicted(doing same stuff as before)
import cv2
import os
import glob
img_dir = '/kaggle/input/captcha-dataset/test_image/test' # Enter Directory of all images 
data_path = os.path.join(img_dir,'*.png')
files = glob.glob(data_path)
data = []    #contain train_img
for f1 in files:
    img = cv2.imread(f1)
    data.append(img)
test_wrds_label=[]   #contains train_img words name corresponding to train_img 
for dirname, _, filenames in os.walk('/kaggle/input/captcha-dataset/test_image/test'):
    for filename in filenames:
        test_wrds_label.append(filename.split(".")[0])

    # len(data)    


In [ ]:
len_wrds=[]
import os
for dirname, _, filenames in os.walk('/kaggle/input/captcha-dataset/test_image/test'):
    for filename in filenames:
        len_wrds.append(len(filename)-4)
print(len_wrds)     

In [ ]:
#LCS algorithm
def lcs( str1, str2, p, q ):
    if p == 0 or q == 0:
        return 0
    elif str1[p-1] == str2[q-1]:
        return 1 + lcs( str1, str2, p - 1, q - 1 )
    else:
        return max( lcs( str1, str2, p - 1, q ), lcs( str1, str2, p, q - 1 ) )

In [ ]:
contours_no=[]
idx=-1
acc_1=0
acc_2=0
total_letters_test=0
for img in data:
    idx=idx+1
    img_rgb=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_gray=cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    _, thresh_img = cv2.threshold(img_gray, 127, 255, cv2.THRESH_BINARY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    morph_img = cv2.morphologyEx(thresh_img, cv2.MORPH_CLOSE, kernel)
    contours, hierarchy = cv2.findContours(
                                   image = morph_img, 
                                   mode = cv2.RETR_TREE, 
                                   method = cv2.CHAIN_APPROX_SIMPLE)
    
    contours_new=[]
    threshold_area = 1300    #threshold area    
    
    for cnt in contours:        
        area = cv2.contourArea(cnt)         
        if area > threshold_area:                   
             contours_new.append(cnt)
    
    # Draw the contour 
    img_copy = img_rgb.copy()
    final = cv2.drawContours(img_copy, contours_new, contourIdx = -1, 
                         color = (255, 0, 0), thickness = 2)

    
    img_copy = img_rgb.copy()
    
    cnt_rect=[]  #contour rectangles
    
    for c in contours_new:
        x,y,w,h=cv2.cv2.boundingRect(c)
        if(w<120 and h>65 and h<110):
            cnt_rect.append((x,y,w,h))
#             img_box = cv2.rectangle(img_copy, (x-5, y-5), (x+w+5, y+h+5), color = (255, 0, 0), thickness = 2)
#     removing the rectangles inside bigger rectangle leaving only the bigges rectangle
    
    rcr=[] #real contour rectangles afterremoving several inner rectangles from outer
    
    for i in cnt_rect:
        flag=0
        for j in cnt_rect:
                if((i[0]>j[0]) and (i[0]+i[2])<(j[0]+j[2]) and (i[1]>j[1]) and (i[1]+i[3])<(j[1]+j[3])):
                    flag=1
                    break
        if flag==0:
            rcr.append(i)
     
    rcr=sorted(rcr, key=lambda a: a[0]) #sorting contour rectangles wrt to x so that we get letters according the order on which they originally are  
    contours_no.append(len(rcr)) 
#     img_copy = img_rgb.copy()
#     for c in rcr:
#         (x,y,w,h)= c
#         img_box = cv2.rectangle(img_copy, (x-10, y-10), (x+w+10, y+h+10), color = (255, 0, 0), thickness = 2)  
    letter_img=[]
    i=0
    str=[]
    s=""
    for box in rcr:
        letter_img.append(img_copy[box[1]:box[1]+box[3],box[0]:box[0]+box[2]])
        img=letter_img[i]
        total_letters_test+=1
#         cv2.imwrite(os.path.join(dirname, test_wrds_label[idx] + str(i)), img)
        i=i+1
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray = cv2.resize(255-gray, (100, 100))
        x_test=np.array(gray)
        x_test = x_test.reshape(1,100, 100,1)
        x_test = x_test.astype('float32')
        x_test = x_test/255
        y_pred=model.predict(x_test)
        y_pred=chr(y_pred.argmax()+65)
        str.append(y_pred)
    s=s.join(str)
#     print(s,test_wrds_label[idx])
    if s==test_wrds_label[idx]:
        acc_1+=1
    
    acc_2=acc_2+lcs(s,test_wrds_label[idx],len(s),len(test_wrds_label[idx]))
    
            
#     fig, axs = plt.subplots(nrows = 1, ncols = len(letter_img), figsize = (20, 20))
#     for i in range(0, len(letter_img) ):
#         ax = axs[i]
#         ax.imshow(letter_img[i])    

In [ ]:
#calculating accuracy according to no of captcha predicted correct
acc_1=acc_1/500*100 
print(acc_1)

In [ ]:
#calculating accuracy according to LCS
acc_2=acc_2/total_letters_test*100
print(acc_2)